In [4]:
import cv2
import numpy as np
import threading
import queue
import time

BUFFER_SIZE = 5  # Number of frames to buffer between producer and consumer
DISPLAY_FPS = 30 # Target FPS for display. Setting this too high might cause stuttering.

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Queues for inter-thread communication
frame_queue = queue.Queue(maxsize=BUFFER_SIZE)
processed_frame_queue = queue.Queue(maxsize=BUFFER_SIZE)

# Event to signal when the video reading is done
video_finished = threading.Event()
processing_finished = threading.Event()

# --- Producer Thread: Reads frames from video ---
def frame_reader(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        video_finished.set()
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_queue.put(frame)
    
    cap.release()
    video_finished.set()
    print("Frame reader finished.")

# --- Consumer Thread: Processes frames ---
def frame_processor():
    while True:
        if video_finished.is_set() and frame_queue.empty():
            break

        try:
            # This prevents the thread from blocking indefinitely if the producer is slow
            # or has finished and there are no more frames.
            frame = frame_queue.get(timeout=0.1) 
        except queue.Empty:
            continue # No frame available, try again

        frame_resized = cv2.resize(frame, (640, 480))

        boxes, weights = hog.detectMultiScale(frame_resized, winStride=(8,8), scale=1.02, padding=(4,4))

        for i, (x,y,w,h) in enumerate(boxes):
            if weights[i] < 0.13:
                continue
            #elif weights[i] < 0.3 and weights[i] > 0.13:
            #    cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (0, 0, 255), 2)
            #elif weights[i] < 0.7 and weights[i] > 0.3:
            #    cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (50, 122, 255), 2)
            elif weights[i] > 0.7:
                cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.putText(frame_resized, 'High', (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        #cv2.putText(frame_resized, 'Moderate', (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50, 122, 255), 2)
        #cv2.putText(frame_resized, 'Low', (10, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        processed_frame_queue.put(frame_resized)
        frame_queue.task_done()
    
    processing_finished.set() # Signal that processing is complete
    print("Frame processor finished.")

def main():
    video_path = '../853889-hd_1920_1080_25fps.mp4'

    # Start the producer thread
    reader_thread = threading.Thread(target=frame_reader, args=(video_path,))
    reader_thread.daemon = True # Allow the main program to exit even if this thread is running
    reader_thread.start()

    # Start the consumer thread
    processor_thread = threading.Thread(target=frame_processor)
    processor_thread.daemon = True
    processor_thread.start()

    last_display_time = time.time()

    while True:
        if video_finished.is_set() and processing_finished.is_set() and processed_frame_queue.empty():
            break
        
        try:
            # Get processed frame for display
            frame_to_display = processed_frame_queue.get(timeout=0.1) 
        except queue.Empty:
            # If the queue is empty, wait a bit before trying again to avoid busy-waiting
            time.sleep(0.01) 
            continue

        cv2.imshow("Frame", frame_to_display)
        processed_frame_queue.task_done() # Indicate that a task from processed_frame_queue is done

        key = cv2.waitKey(1)
        if key == 27: # ESC key to exit
            break

    print("Exiting main display loop.")

    cv2.destroyAllWindows()
    print("Program finished.")

if __name__ == "__main__":
    main()

Frame reader finished.
Frame processor finished.
Exiting main display loop.
Program finished.
